In [1]:
!pip install langchain chromadb beautifulsoup4 requests langchain-openai langchain-ollama

In [4]:
import os
# 환경 변수 설정
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력
os.environ["USER_AGENT"] = "MyCustomUserAgent/1.0"

from langchain_ollama import OllamaLLM
# DeepSeek R1 모델 로드
deepseek = OllamaLLM(model="deepseek-r1")

In [5]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

# 웹 크롤링하여 텍스트 데이터 가져오기
url = "https://ko.wikipedia.org/wiki/%EA%B2%80%EC%83%89%EC%A6%9D%EA%B0%95%EC%83%9D%EC%84%B1"  # AI 관련 위키 문서
loader = WebBaseLoader(url) # 웹 페이지의 내용을 가져와 랭체인 문서 객체로 변환
documents = loader.load() # 웹 페이지에서 텍스트 데이터를 추출

# 텍스트를 작은 단위로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) # 웹에서 가져온 문서(documents)를 500자 단위로 분할
split_docs = text_splitter.split_documents(documents)

# OpenAI Embeddings로 벡터 변환
embeddings = OpenAIEmbeddings()

# ChromaDB 벡터 저장소 생성 및 데이터 저장
db = Chroma.from_documents(split_docs, embeddings, persist_directory="./chroma_db") # 분할된 문서를 벡터화하여 ChromaDB에 저장, 재사용 가능하도록 로컬 디스크에 저장

# ChromaDB에서 검색할 retriever 생성
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5}) # 유사한 문서를 검색, 최대 5개의 유사한 검색 결과 반환

# LangChain Q&A 체인 설정
qa_chain = RetrievalQA.from_chain_type(llm=deepseek, retriever=retriever, chain_type="stuff") # 검색된 문서를 기반으로 LLM이 답변 생성

# 사용자 질문 입력, 결과를 반환을 위한 검색 및 답변 생성
query = "RAG란?"
response = qa_chain.invoke(query)

print("📌 답변:")
print(response)

📌 답변:
{'query': 'RAG란?', 'result': "<think>\nOkay, I need to explain what RAG is based on the provided context. Let me read through it carefully.\n\nSo, RAG stands for Retrieval-Augmented Generation. It's a technique that improves the performance of large language models by using a database during their interaction. The main idea is that when someone asks a question, instead of relying solely on the model's internal knowledge, it can search an external database to get relevant information first.\n\nFrom the context, I see that RAG modifies how LLMs interact with users by adding this retrieval aspect. It helps the model use up-to-date and domain-specific information more effectively. The examples given include chatbots accessing company data or using reliable sources only.\n\nI should make sure to explain each part clearly: what RAG is, how it works by combining model generation with database searching, its benefits, and some examples of where it's applied.\n</think>\n\nRAG (Retrieval-A

In [6]:
query = "RAG 사용 사례는?"
response = qa_chain.invoke(query)

print("📌 답변:")
print(response)

📌 답변:
{'query': 'RAG 사용 사례는?', 'result': '<think>\nOkay, so I need to figure out the answer to the question about RAG usage cases based on the provided context. Let me read through the context carefully.\n\nFirst, the context starts by explaining what RAG is—Search-Augmented Generation using large language models (LLMs) and retrieved information from a database. It mentions that RAG improves LLM performance by integrating external data during the generation process.\n\nLooking further down, there are multiple sections about usage cases. The first one I notice says: "使用事例包含了回公司内部数据支持的聊天机器人访问权限提供，或者仅从可靠来源获取的事实信息的聊天机器人包含。" So that translates to usage examples include providing chatbot access to internal company data or using only trustworthy sources for factual information in the chatbot.\n\nAnother usage case mentioned is enhancing search engines by improving their ability to answer questions through augmented generation. That\'s another important point because RAG can make search more e